Paleta de colores
['#f5f5f1', '#e50914', '#b20710', '#221f1f']


In [7]:
import pandas as pd
from openpyxl.utils import get_column_letter

src = "netflix.csv"
normalized = "netflix_normalized.csv"
out = "netflix.xlsx"

# Normalizar: ';' -> ',' solo cuando NO está dentro de comillas
def normalize_delimiters(in_path, out_path):
    with open(in_path, "r", encoding="utf-8-sig", newline="") as fin, \
         open(out_path, "w", encoding="utf-8", newline="") as fout:
        in_quotes = False
        while True:
            ch = fin.read(1)
            if not ch:
                break
            if ch == '"':
                in_quotes = not in_quotes
                fout.write(ch)
            elif ch == ';' and not in_quotes:
                fout.write(',')
            else:
                fout.write(ch)

normalize_delimiters(src, normalized)

# Leer CSV ya normalizado (coma), respetando comillas y espacios
df = pd.read_csv(
    normalized,
    sep=",",
    quotechar='"',
    skipinitialspace=True,
    engine="python",
    on_bad_lines="warn"
)

# Limpiar
df.columns = df.columns.str.strip()
df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
df = df.dropna(axis=1, how="all")

# Exportar a Excel con autoajuste
with pd.ExcelWriter(out, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="Datos")
    ws = writer.sheets["Datos"]
    for i, col in enumerate(df.columns, start=1):
        max_len = max(len(str(col)), df[col].astype(str).map(len).max() if not df.empty else len(str(col)))
        ws.column_dimensions[get_column_letter(i)].width = max(10, min(max_len + 2, 60))


In [8]:
# Mostrar las primeras filas y los nombres de las columnas para revisar el formato del CSV
print("Nombres de columnas:", df.columns.tolist())
display(df.head())

Nombres de columnas: ['show_id', 'type', 'title', 'cast', 'country', 'release_year', 'rating', 'duration', 'listed_in', 'description']


,show_id,type,title,cast,country,release_year,rating,duration,listed_in,description
0,"s1,TV Show,3%,,""João Miguel, Bianca Comparato,...",NaN,NaN,None,None,NaN,None,None,None,None
1,"s2,Movie,7:19,Jorge Michel Grau,""Demián Bichir...",NaN,NaN,None,None,NaN,None,None,None,None
2,"s3,Movie,23:59,Gilbert Chan,""Tedd Chan, Stella...",NaN,NaN,None,None,NaN,None,None,None,None
3,"s4,Movie,9,Shane Acker,""Elijah Wood, John C. R...",NaN,NaN,None,None,NaN,None,None,None,None
4,"s5,Movie,21,Robert Luketic,""Jim Sturgess, Kevi...",NaN,NaN,None,None,NaN,None,None,None,None
